# The Diffusion Equation

The diffusion equation is relatively straightforward to express as a partial differential equation comparing concentration with time:

$$ \frac{dc}{dt} = D\nabla^2c $$

There are several standard ways to discretize it:

$\cdot$ Explicit

$\cdot$ Implicit

$\cdot$ Crank-Nicolson

There is also the ADI method for Implicit and Crank-Nicolson schemes for multiple dimensions, allowing for solutions only involving standard tridiagonal matrices (or thereabouts).

Additionally, rather than inverting the matrix or exactly solving for the value of the vector x where Ax=b, an approximate solution method can be used to speed up solutions: the Generalized Minimal Residuals (GMRES) method, for example, is one of them.

## pyphasefield

pyphasefield is a python package that may be downloaded from PyPI using the command `pip install pyphasefield`. The current version of the code (as of writing this!) is 0.0.15 and works for the following examples below!

## Solution Specifics

Here we will write out the exact manner by which each of these Diffusion examples can be run in python using the most recent version of pyphasefield (0.0.15). This will unfortunately break after the rewrite to avoid the God Object "Simulation", so we have included a self-contained version of the pyphasefield code to go along with this notebook.

### Explicit Diffusion

This method is the most straightforward: future values are computed directly from the previous iteration, using the equation (in 1d):

$$ \frac{c_x^{t+1} - c_x^t}{\Delta t} = D\frac{c_{x+1}^t + c_{x-1}^t - 2c_{x}^t}{\Delta x^2} $$

This equation can be expanded to cover multiple dimensions, for example in 2d:

$$ \frac{c_{x,y}^{t+1} - c_{x,y}^t}{\Delta t} = D\frac{c_{x+1,y}^t + c_{x-1,y}^t + c_{x,y+1}^t + c_{x,y-1}^t - 4c_{x,y}^t}{\Delta x^2} $$

And in 3d:

$$ \frac{c_{x,y,z}^{t+1} - c_{x,y,z}^t}{\Delta t} = D\frac{c_{x+1,y,z}^t + c_{x-1,y,z}^t + c_{x,y+1,z}^t + c_{x,y-1,z}^t + c_{x,y,z+1}^t + c_{x,y,z-1}^t - 6c_{x,y,z}^t}{\Delta x^2} $$

For future examples (other than for ADI where it is crucial to explain in detail), we will not show the higher dimensional examples. What is important to know is that the stencil can be expanded to multiple dimensions by including the relevant neighbor terms in that new dimension, and increasing the coefficient on the central cell in the right hand side by 2 (for example: -2 for 1d, -4 for 2d, -6 for 3d, as seen above).

In [ ]:
import numpy as np
import pyphasefield as ppf
import matplotlib.pyplot as plt

"""
1d explicit diffusion example
"""

size1d = [100]
sim_explicit1d = ppf.Simulation("Explicit1D")
sim_explicit1d.init_sim_Diffusion(dim=size1d, solver="explicit")
c = np.zeros(size1d)
c[size1d[0]//2] = 1
sim_explicit1d.fields[0].data = c.copy()

sim_explicit1d.simulate(100)

"""
2d explicit diffusion example
"""

size2d = [11, 11]
sim_explicit2d = ppf.Simulation("Explicit2D")
sim_explicit2d.init_sim_Diffusion(dim=size2d, solver="explicit")
c = np.zeros(size2d)
c[size2d[0]//2, size2d[1]//2] = 1
sim_explicit2d.fields[0].data = c.copy()

sim_explicit2d.simulate(100)

"""
3d explicit diffusion example
"""

size3d = [7, 7, 7]
sim_explicit3d = ppf.Simulation("Explicit2D")
sim_explicit3d.init_sim_Diffusion(dim=size2d, solver="explicit")
c = np.zeros(size3d)
c[size3d[0]//2, size3d[1]//2, size3d[2]//2] = 1
sim_explicit3d.fields[0].data = c.copy()

sim_explicit3d.simulate(100)

"""
plot results (1d plot for 1d, imshow for 2d, imshow slice for 3d)
"""

x = np.arange(0, size1d[0])
plt.plot(x, sim_explicit1d.fields[0].data, 'k')
plt.show()

plt.imshow(sim_explicit2d.fields[0].data, interpolation="nearest")
plt.show()

plt.imshow(sim_explicit3d.fields[0].data[3], interpolation="nearest")
plt.show()

### Implicit and Crank-Nicolson Diffusion in 1D

This method is more complicated, as previously known values are computed from the future values. Therefore, in order to get the future values, we are required to solve the inverse problem by inverting the equations matrix or an equivalent method. The advantage of doing this is that this method is stable for every time step, unlike the explicit scheme which is only stable below the Courant condition. The scheme for Implicit is as follows:

$$ \frac{c_x^{t+1} - c_x^t}{\Delta t} = D\frac{c_{x+1}^{t+1} + c_{x-1}^{t+1} - 2c_{x}^{t+1}}{\Delta x^2} $$

Crank-Nicolson effectively computes an average of the two schemes. This produces a scheme which is slightly more complicated than a purely implicit scheme, but is more accurate (second-order accurate in both time and space!). The discretization equation for the Crank-Nicolson method is given by:

$$ \frac{c_x^{t+1} - c_x^t}{\Delta t} = \frac{D}{2}\frac{c_{x+1}^t + c_{x-1}^t - 2c_{x}^t + c_{x+1}^{t+1} + c_{x-1}^{t+1} - 2c_{x}^{t+1}}{\Delta x^2} $$

In both cases, the equation is solved at each timestep by separating out the equation in terms of present and future cell values, converting these equations into matrix notation, then solving the resultant equation, Ax = b, by some method. Here's how this is done for the Implicit scheme:

$$ \frac{c_x^{t+1} - c_x^t}{\Delta t} = D\frac{c_{x+1}^{t+1} + c_{x-1}^{t+1} - 2c_{x}^{t+1}}{\Delta x^2} $$

$$ c_x^{t+1} - c_x^t = \frac{D\Delta t}{\Delta x^2}(c_{x+1}^{t+1} + c_{x-1}^{t+1} - 2c_{x}^{t+1}) $$

$$ \frac{D\Delta t}{\Delta x^2} = \alpha $$

$$ c_x^{t+1} - c_x^t = \alpha(c_{x+1}^{t+1} + c_{x-1}^{t+1} - 2c_{x}^{t+1}) $$

$$ (1+2\alpha)c_x^{t+1} -\alpha c_{x+1}^{t+1} - \alpha c_{x-1}^{t+1} = c_x^t $$

By combining this equation with all the other equations for each cell in the 1d array, and assuming periodic boundary conditions, we get the following matrix equation for an example problem of 5 cells:

$$ \begin{bmatrix}
    1+2\alpha & -\alpha & 0 & 0 & -\alpha\\
    -\alpha & 1+2\alpha & -\alpha & 0 & 0\\
    0 & -\alpha & 1+2\alpha & -\alpha & 0\\
    0 & 0 & -\alpha & 1+2\alpha & -\alpha\\
    -\alpha & 0 & 0 & -\alpha & 1+2\alpha\\
    \end{bmatrix} \begin{bmatrix}
    c_{0}^{t+1}\\
    c_{1}^{t+1}\\
    c_{2}^{t+1}\\
    c_{3}^{t+1}\\
    c_{4}^{t+1}
    \end{bmatrix} = \begin{bmatrix}
    c_{0}^{t}\\
    c_{1}^{t}\\
    c_{2}^{t}\\
    c_{3}^{t}\\
    c_{4}^{t}
    \end{bmatrix}$$
    
This equation of the form Ax=b can be solved by either inverting the matrix (or some equivalent exact solution) or using an approximate method like GMRES. Both examples will be shown below

These methods can be directly extended to 2d and 3d by flattening the concentration arrays, and using an appropriately designed (but complicated!) matrix

In [ ]:
import numpy as np
import pyphasefield as ppf
import matplotlib.pyplot as plt

size1d = [100]
c = np.zeros(size1d)
c[size1d[0]//2] = 1

"""
1d implicit diffusion example
"""

sim_implicit1d = ppf.Simulation("Implicit1D")
sim_implicit1d.init_sim_Diffusion(dim=size1d, solver="implicit")
sim_implicit1d.fields[0].data = c.copy()

sim_implicit1d.simulate(100)

"""
1d implicit diffusion example, using a GMRES approximate solution
"""

sim_implicit1d_gmres = ppf.Simulation("Implicit1D_GMRES")
sim_implicit1d_gmres.init_sim_Diffusion(dim=size1d, solver="implicit", gmres=True)
sim_implicit1d_gmres.fields[0].data = c.copy()

sim_implicit1d_gmres.simulate(100)

"""
1d Crank-Nicolson diffusion example
"""

sim_cn1d = ppf.Simulation("CrankNicolson1D")
sim_cn1d.init_sim_Diffusion(dim=size1d, solver="crank-nicolson")
sim_cn1d.fields[0].data = c.copy()

sim_cn1d.simulate(100)

"""
1d Crank-Nicolson diffusion example, using a GMRES approximate solution
"""

sim_cn1d_gmres = ppf.Simulation("CrankNicolson1D_GMRES")
sim_cn1d_gmres.init_sim_Diffusion(dim=size1d, solver="crank-nicolson", gmres=True)
sim_cn1d_gmres.fields[0].data = c.copy()

sim_cn1d_gmres.simulate(100)

"""
plot results
"""

x = np.arange(0, size1d[0])
plt.plot(x, sim_implicit1d.fields[0].data, 'r')
plt.plot(x, sim_implicit1d_gmres.fields[0].data, 'g')
plt.plot(x, sim_cn1d.fields[0].data, 'b')
plt.plot(x, sim_cn1d_gmres.fields[0].data, 'k')
plt.show()

plt.plot(x, sim_implicit1d.fields[0].data, 'r')
plt.plot(x, sim_implicit1d_gmres.fields[0].data-0.1, 'g')
plt.plot(x, sim_cn1d.fields[0].data-0.2, 'b')
plt.plot(x, sim_cn1d_gmres.fields[0].data-0.3, 'k')
plt.show()

### ADI Methods

The matrix in the matrix equations above is a square matrix with length equal to the product of the dimensions of the simulation region: a 1000^3 simulation would require a matrix of size 10^9. Matrix inversion is an O(n^3) operation, with n being the length of a side of the matrix, so inverting this matrix would require on the order of 10^27 operations. On a traditional personal computer with 1 GFLOP of processing power, this would be doable if you happen to have a spare 31 billion years. For the rest of us, such a task is impossible.

Instead of this, we approximate the discretization by independent, 1D steps. For a 2D implicit scheme, this becomes:

$$ \frac{c_{x,y}^{*} - c_{x,y}^t}{\Delta t} = D\frac{c_{x+1,y}^{*} + c_{x-1,y}^{*} - 2c_{x,y}^{*}}{\Delta x^2} $$

$$ \frac{c_{x,y}^{t+1} - c_{x,y}^*}{\Delta t} = D\frac{c_{x,y+1}^{t+1} + c_{x,y-1}^{t+1} - 2c_{x,y}^{t+1}}{\Delta x^2} $$

In this setup, all values with an asterisk are intermediate values. Be careful, the values at these intermediate timesteps do not necessarily respect the diffusion equation!

For 3D, we add an additional intermediate step:

$$ \frac{c_{x,y,z}^{*} - c_{x,y,z}^t}{\Delta t} = D\frac{c_{x+1,y,z}^{*} + c_{x-1,y,z}^{*} - 2c_{x,y,z}^{*}}{\Delta x^2} $$

$$ \frac{c_{x,y,z}^{**} - c_{x,y,z}^*}{\Delta t} = D\frac{c_{x,y+1,z}^{**} + c_{x,y-1,z}^{**} - 2c_{x,y,z}^{**}}{\Delta x^2} $$

$$ \frac{c_{x,y,z}^{t+1} - c_{x,y,z}^{**}}{\Delta t} = D\frac{c_{x,y,z+1}^{t+1} + c_{x,y,z-1}^{t+1} - 2c_{x,y,z}^{t+1}}{\Delta x^2} $$

For Crank-Nicolson equations in ADI, pyphasefield uses the Peaceman-Rachford method for computing the steps. In this, an explicit step in x is combined with an implicit step in y for the intermediate step, and an explicit step in y is combined with an implicit step in x for the final half:

$$ \frac{c_{x,y}^{*} - c_{x,y}^t}{\Delta t} = \frac{D}{2}\frac{c_{x+1,y}^{t} + c_{x-1,y}^{t} - 2c_{x,y}^{t} + c_{x,y+1}^{*} + c_{x,y-1}^{*} - 2c_{x,y}^{*}}{\Delta x^2} $$

$$ \frac{c_{x,y}^{t+1} - c_{x,y}^*}{\Delta t} = \frac{c_{x+1,y}^{*} + c_{x-1,y}^{*} - 2c_{x,y}^{*} + c_{x,y+1}^{t+1} + c_{x,y-1}^{t+1} - 2c_{x,y}^{t+1}}{\Delta x^2} $$

We extend this to 3d using the following steps for Crank-Nicolson 3D ADI:

Explicit x, Implicit y

Explicit y, Implicit z

Explicit z, Implicit x

$$ \frac{c_{x,y,z}^{*} - c_{x,y,z}^t}{\Delta t} = \frac{D}{2}\frac{c_{x+1,y,z}^{t} + c_{x-1,y,z}^{t} - 2c_{x,y,z}^{t} + c_{x,y+1,z}^{*} + c_{x,y-1,z}^{*} - 2c_{x,y,z}^{*}}{\Delta x^2} $$

$$ \frac{c_{x,y,z}^{**} - c_{x,y,z}^*}{\Delta t} = \frac{D}{2}\frac{c_{x,y+1,z}^{*} + c_{x,y-1,z}^{*} - 2c_{x,y,z}^{*} + c_{x,y,z+1}^{**} + c_{x,y,z-1}^{**} - 2c_{x,y,z}^{**}}{\Delta x^2} $$

$$ \frac{c_{x,y,z}^{t+1} - c_{x,y,z}^{**}}{\Delta t} = \frac{D}{2}\frac{c_{x,y,z+1}^{**} + c_{x,y,z-1}^{**} - 2c_{x,y,z}^{**} + c_{x+1,y,z}^{t+1} + c_{x-1,y,z}^{t+1} - 2c_{x,y,z}^{t+1}}{\Delta x^2} $$

The code below show two snippets, one without ADI and one with ADI. Compare the runtime!

In [ ]:
import numpy as np
import pyphasefield as ppf
import matplotlib.pyplot as plt

size2d = [50, 50]
c = np.zeros(size2d)
c[size2d[0]//2, size2d[1]//2] = 1

"""
2d implicit diffusion example
"""

sim_implicit2d = ppf.Simulation("Implicit2D")
sim_implicit2d.init_sim_Diffusion(dim=size2d, solver="implicit")
sim_implicit2d.fields[0].data = c.copy()

for i in range(10):
    sim_implicit2d.simulate(10)
    print("Implicit is "+str(i+1)+"0% done!")

"""
2d crank-nicolson diffusion example
"""

sim_cn2d = ppf.Simulation("CrankNicolson2D")
sim_cn2d.init_sim_Diffusion(dim=size2d, solver="crank-nicolson")
sim_cn2d.fields[0].data = c.copy()

for i in range(10):
    sim_cn2d.simulate(10)
    print("Crank-Nicolson is "+str(i+1)+"0% done!")

"""
plot results
"""

plt.imshow(sim_implicit2d.fields[0].data)
plt.show()
plt.imshow(sim_cn2d.fields[0].data)
plt.show()
print(np.max(sim_implicit2d.fields[0].data))
print(np.max(sim_cn2d.fields[0].data))

In [ ]:
import numpy as np
import pyphasefield as ppf
import matplotlib.pyplot as plt

size2d = [50, 50]
c = np.zeros(size2d)
c[size2d[0]//2, size2d[1]//2] = 1

"""
2d implicit ADI diffusion example
"""

sim_implicit2d = ppf.Simulation("Implicit2D_ADI")
sim_implicit2d.init_sim_Diffusion(dim=size2d, solver="implicit", adi=True)
sim_implicit2d.fields[0].data = c.copy()

for i in range(10):
    sim_implicit2d.simulate(10)
    print("Implicit is "+str(i+1)+"0% done!")

"""
2d crank-nicolson ADI diffusion example
"""

sim_cn2d = ppf.Simulation("CrankNicolson2D_ADI")
sim_cn2d.init_sim_Diffusion(dim=size2d, solver="crank-nicolson", adi=True)
sim_cn2d.fields[0].data = c.copy()

for i in range(10):
    sim_cn2d.simulate(10)
    print("Crank-Nicolson is "+str(i+1)+"0% done!")

"""
plot results
"""

plt.imshow(sim_implicit2d.fields[0].data)
plt.show()
plt.imshow(sim_cn2d.fields[0].data)
plt.show()
print(np.max(sim_implicit2d.fields[0].data))
print(np.max(sim_cn2d.fields[0].data))

### Implementation of all possible Diffusion simulations

Below is a code block that just contains the initialization of all diffusion examples in pyphasefield, for an arbitrary size for 1d, 2d, and 3d simulations. Running the following code block will not actually do anything except initialize these Simulation objects. These can be run by copying these to a code block and calling \[sim_name\].simulate(100) to simulate for 100 timesteps, for instance

In [ ]:
import numpy as np
import pyphasefield as ppf
import matplotlib.pyplot as plt

"""
Explicit 1D
"""

size1d = [100]
c_1d = np.zeros(size1d)
c_1d[50] = 1
sim_Explicit1D = ppf.Simulation("Explicit1D")
sim_Explicit1D.init_sim_Diffusion(size1d, solver="explicit")
sim_Explicit1D.fields[0].data = c_1d.copy()

"""
Explicit 2D
"""

size2d = [11, 11]
c_2d = np.zeros(size2d)
c_2d[5, 5] = 1
sim_Explicit2D = ppf.Simulation("Explicit2D")
sim_Explicit2D.init_sim_Diffusion(size2d, solver="explicit")
sim_Explicit2D.fields[0].data = c_2d.copy()

"""
Explicit 3D
"""

size3d = [7, 7, 7]
c_3d = np.zeros(size3d)
c_3d[3, 3, 3] = 1
sim_Explicit3D = ppf.Simulation("Explicit3D")
sim_Explicit3D.init_sim_Diffusion(size3d, solver="explicit")
sim_Explicit3D.fields[0].data = c_3d.copy()

"""
Implicit 1D
"""

size1d = [100]
c_1d = np.zeros(size1d)
c_1d[50] = 1
sim_Implicit1D = ppf.Simulation("Implicit1D")
sim_Implicit1D.init_sim_Diffusion(size1d, solver="implicit")
sim_Implicit1D.fields[0].data = c_1d.copy()

"""
Implicit 1D, GMRES approximation
"""

size1d = [100]
c_1d = np.zeros(size1d)
c_1d[50] = 1
sim_Implicit1D_GMRES = ppf.Simulation("Implicit1D_GMRES")
sim_Implicit1D_GMRES.init_sim_Diffusion(size1d, solver="implicit", gmres=True)
sim_Implicit1D_GMRES.fields[0].data = c_1d.copy()

"""
Implicit 2D
"""

size2d = [11, 11]
c_2d = np.zeros(size2d)
c_2d[5, 5] = 1
sim_Implicit2D = ppf.Simulation("Implicit2D")
sim_Implicit2D.init_sim_Diffusion(size2d, solver="implicit")
sim_Implicit2D.fields[0].data = c_2d.copy()

"""
Implicit 2D, GMRES approximation
"""

size2d = [11, 11]
c_2d = np.zeros(size2d)
c_2d[5, 5] = 1
sim_Implicit2D_GMRES = ppf.Simulation("Implicit2D_GMRES")
sim_Implicit2D_GMRES.init_sim_Diffusion(size2d, solver="implicit", gmres=True)
sim_Implicit2D_GMRES.fields[0].data = c_2d.copy()

"""
Implicit 3D
"""

size3d = [7, 7, 7]
c_3d = np.zeros(size3d)
c_3d[3, 3, 3] = 1
sim_Implicit3D = ppf.Simulation("Implicit3D")
sim_Implicit3D.init_sim_Diffusion(size3d, solver="implicit")
sim_Implicit3D.fields[0].data = c_3d.copy()

"""
Implicit 3D, GMRES approximation
"""

size3d = [7, 7, 7]
c_3d = np.zeros(size3d)
c_3d[3, 3, 3] = 1
sim_Implicit3D_GMRES = ppf.Simulation("Implicit3D_GMRES")
sim_Implicit3D_GMRES.init_sim_Diffusion(size3d, solver="implicit", gmres=True)
sim_Implicit3D_GMRES.fields[0].data = c_3d.copy()

"""
Implicit 2D, ADI
"""

size2d = [11, 11]
c_2d = np.zeros(size2d)
c_2d[5, 5] = 1
sim_Implicit2D_ADI = ppf.Simulation("Implicit2D_ADI")
sim_Implicit2D_ADI.init_sim_Diffusion(size2d, solver="implicit", adi=True)
sim_Implicit2D_ADI.fields[0].data = c_2d.copy()

"""
Implicit 2D, ADI, GMRES approximation
"""

size2d = [11, 11]
c_2d = np.zeros(size2d)
c_2d[5, 5] = 1
sim_Implicit2D_ADI_GMRES = ppf.Simulation("Implicit2D_ADI_GMRES")
sim_Implicit2D_ADI_GMRES.init_sim_Diffusion(size2d, solver="implicit", gmres=True, adi=True)
sim_Implicit2D_ADI_GMRES.fields[0].data = c_2d.copy()

"""
Implicit 3D, ADI
"""

size3d = [7, 7, 7]
c_3d = np.zeros(size3d)
c_3d[3, 3, 3] = 1
sim_Implicit3D_ADI = ppf.Simulation("Implicit3D_ADI")
sim_Implicit3D_ADI.init_sim_Diffusion(size3d, solver="implicit", adi=True)
sim_Implicit3D_ADI.fields[0].data = c_3d.copy()

"""
Implicit 3D, ADI, GMRES approximation
"""

size3d = [7, 7, 7]
c_3d = np.zeros(size3d)
c_3d[3, 3, 3] = 1
sim_Implicit3D_ADI_GMRES = ppf.Simulation("Implicit3D_ADI_GMRES")
sim_Implicit3D_ADI_GMRES.init_sim_Diffusion(size3d, solver="implicit", gmres=True, adi=True)
sim_Implicit3D_ADI_GMRES.fields[0].data = c_3d.copy()

"""
Crank-Nicolson 1D
"""

size1d = [100]
c_1d = np.zeros(size1d)
c_1d[50] = 1
sim_CrankNicolson1D = ppf.Simulation("CrankNicolson1D")
sim_CrankNicolson1D.init_sim_Diffusion(size1d, solver="crank-nicolson")
sim_CrankNicolson1D.fields[0].data = c_1d.copy()

"""
Crank-Nicolson 1D, GMRES approximation
"""

size1d = [100]
c_1d = np.zeros(size1d)
c_1d[50] = 1
sim_CrankNicolson1D_GMRES = ppf.Simulation("CrankNicolson1D_GMRES")
sim_CrankNicolson1D_GMRES.init_sim_Diffusion(size1d, solver="crank-nicolson", gmres=True)
sim_CrankNicolson1D_GMRES.fields[0].data = c_1d.copy()

"""
Crank-Nicolson 2D
"""

size2d = [11, 11]
c_2d = np.zeros(size2d)
c_2d[5, 5] = 1
sim_CrankNicolson2D = ppf.Simulation("CrankNicolson2D")
sim_CrankNicolson2D.init_sim_Diffusion(size2d, solver="crank-nicolson")
sim_CrankNicolson2D.fields[0].data = c_2d.copy()

"""
Crank-Nicolson 2D, GMRES approximation
"""

size2d = [11, 11]
c_2d = np.zeros(size2d)
c_2d[5, 5] = 1
sim_CrankNicolson2D_GMRES = ppf.Simulation("CrankNicolson2D_GMRES")
sim_CrankNicolson2D_GMRES.init_sim_Diffusion(size2d, solver="crank-nicolson", gmres=True)
sim_CrankNicolson2D_GMRES.fields[0].data = c_2d.copy()

"""
Crank-Nicolson 3D
"""

size3d = [7, 7, 7]
c_3d = np.zeros(size3d)
c_3d[3, 3, 3] = 1
sim_CrankNicolson3D = ppf.Simulation("CrankNicolson3D")
sim_CrankNicolson3D.init_sim_Diffusion(size3d, solver="crank-nicolson")
sim_CrankNicolson3D.fields[0].data = c_3d.copy()

"""
Crank-Nicolson 3D, GMRES approximation
"""

size3d = [7, 7, 7]
c_3d = np.zeros(size3d)
c_3d[3, 3, 3] = 1
sim_CrankNicolson3D_GMRES = ppf.Simulation("CrankNicolson3D_GMRES")
sim_CrankNicolson3D_GMRES.init_sim_Diffusion(size3d, solver="crank-nicolson", gmres=True)
sim_CrankNicolson3D_GMRES.fields[0].data = c_3d.copy()

"""
Crank-Nicolson 2D, ADI
"""

size2d = [11, 11]
c_2d = np.zeros(size2d)
c_2d[5, 5] = 1
sim_CrankNicolson2D_ADI = ppf.Simulation("CrankNicolson2D_ADI")
sim_CrankNicolson2D_ADI.init_sim_Diffusion(size2d, solver="crank-nicolson", adi=True)
sim_CrankNicolson2D_ADI.fields[0].data = c_2d.copy()

"""
Crank-Nicolson 2D, ADI, GMRES approximation
"""

size2d = [11, 11]
c_2d = np.zeros(size2d)
c_2d[5, 5] = 1
sim_CrankNicolson2D_ADI_GMRES = ppf.Simulation("CrankNicolson2D_ADI_GMRES")
sim_CrankNicolson2D_ADI_GMRES.init_sim_Diffusion(size2d, solver="crank-nicolson", gmres=True, adi=True)
sim_CrankNicolson2D_ADI_GMRES.fields[0].data = c_2d.copy()

"""
Crank-Nicolson 3D, ADI
"""

size3d = [7, 7, 7]
c_3d = np.zeros(size3d)
c_3d[3, 3, 3] = 1
sim_CrankNicolson3D_ADI = ppf.Simulation("CrankNicolson3D_ADI")
sim_CrankNicolson3D_ADI.init_sim_Diffusion(size3d, solver="crank-nicolson", adi=True)
sim_CrankNicolson3D_ADI.fields[0].data = c_3d.copy()

"""
Crank-Nicolson 3D, ADI, GMRES approximation
"""

size3d = [7, 7, 7]
c_3d = np.zeros(size3d)
c_3d[3, 3, 3] = 1
sim_CrankNicolson3D_ADI_GMRES = ppf.Simulation("CrankNicolson3D_ADI_GMRES")
sim_CrankNicolson3D_ADI_GMRES.init_sim_Diffusion(size3d, solver="crank-nicolson", gmres=True, adi=True)
sim_CrankNicolson3D_ADI_GMRES.fields[0].data = c_3d.copy()